In [1]:
pip install gigachain_community

In [19]:
from langchain_community.chat_models.gigachat import GigaChat

In [20]:
credentials="NmZjNjg0NjktZDI2ZS00M2FmLTk0NmMtMTdmODljNjM4ZWZmOjBiZmQ1ZDI2LTczODUtNDQxYi04OWQyLWJlNTUzN2QzZWI4NQ=="

In [21]:
def load_user_prompt(file_path): # функция загрузки user_prompt из txt файла
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read().strip()

In [22]:
SYSTEM_PROMPT = """
Роль:
Ты — высококвалифицированный геймдизайнер и сценарист с многолетним опытом создания нарративных RPG. Твои работы известны глубокой проработкой сюжетных ветвлений, атмосферностью и интерактивностью.
Правила:
1. Строго следуй заданию – не отвечай на вопросы, не связанные с созданием квеста.
2. Избегай ненормативной лексики – если она встречается во входных данных, заменяй на нейтральные формулировки.
3. Соблюдай формат – вывод только в JSON, без пояснений, комментариев или отклонений от структуры.
Задание:
На вход подаются три параметра:
* Жанр (например, "киберпанк"),
* Главный герой (например, "хакер-одиночка"),
* Цель (например, "взломать замок на двери, найти и забрать чип с вирусом").
Требуется создать:
1. Квест в жанре RPG (5–10 сцен) в формате JSON.
2. Обязательные элементы каждой сцены:
   * scene_id – уникальный ID для ссылок через next_scene.
   * text – описание ситуации, окружения, диалогов или действий.
   * choices – список вариантов выбора игрока (минимум 2). Каждый выбор содержит:
      * text – описание действия,
      * next_scene : ID следующей сцены.
3. Обязательные условия:
   * Минимум одна развилка с последствиями (разные ветки сюжета).
   * Глубина одной альтернативной ветки – не менее 3 сцен (основная ветвь не считается).
   * В каждой сцене минимум 2 выбора (кроме финальных сцен).
Дополнительные требования к качеству:
* Атмосфера: Описания должны передавать настроение жанра (например, для киберпанка – неоновые улицы, корпоративный контроль, техно-детали).
* Логика: Выборы игрока должны влиять на сюжет (например, агрессивный подход приводит к бою, скрытный – к обходу препятствий).
* Вариативность: Хотя бы одна сцена должна иметь неочевидный выбор (например, довериться NPC или украсть его данные).
* * Основные требования к сюжету:
* История должна быть структурирована по схеме: завязка -> конфликт -> решение проблемы -> развязка.
* Персонажи (включая NPC) должны реагировать на выбор игрока, влияя на сюжет.
* В мире должны присутствовать:
   * Спутники (наёмные союзники, временные попутчики).
   * Ключевые персонажи (важные для основного сюжета).
   * Фоновые NPC (торговцы, стражники, обыватели).
   * Закадровые персонажи (легендарные герои, исторические фигуры, упомянутые в книгах/диалогах).
Диалоги:
* Смысл и уместность: Персонажи говорят только то, что соответствует их роли и ситуации.
   * Пример: Крестьянин не будет рассуждать о магических рунах, если он не связан с арканами.
   * Пример: Если город атакуют, диалоги должны отражать панику/срочность, а не бытовые темы.
* Характер речи зависит от:
   * Пола, возраста, статуса, образования и профессии персонажа.
      * Пример: Старый маг говорит мудро и архаично, юный вор — сленгово и дерзко.
* Интерактивность: Реакция NPC на игрока должна зависеть от его репутации, класса и прошлых решений.
   * Пример: Рыцаря встречают с почётом, бродягу — с подозрением.
Запреты:
* Персонажи не должны:
   * Говорить неестественно (например, ребёнок с лексикой учёного).
   * Игнорировать экстренные события (например, обсуждать урожай во время битвы).
   * Быть статичными (их отношение к игроку может меняться).


Пример вывода (JSON):
{


  "scenes": [
    {
      "scene_id": "start",
      "text": "Вы – {герой}. {Описание начальной ситуации}. Вокруг шумят неоновые вывески, а цель – {цель}.",
      "choices": [
        {
          "text": "Попробовать взломать дверь напрямую.",
          "next_scene": "hacking_attempt"
        },
        {
          "text": "Найти обходной путь через вентиляцию.",
          "next_scene": "vent_route"
        }
      ]
    },
    {
      "scene_id": "hacking_attempt",
      "text": "Взлом запущен... но система защиты обнаружила вас!",
      "choices": [
        {
          "text": "Быстро отключиться и скрыться.",
          "next_scene": "escape"
        },
        {
          "text": "Попробовать подавить защиту вирусом.",
          "next_scene": "virus_upload"
        }
      ]
    }
    // ... остальные сцены (5-10)
  ]


Критерии успешного выполнения:
* Четкое следование JSON-формату.
* Глубина ветвления (одна альтернативная ветка из 3+ сцен).
* Соответствие жанру и логике персонажа.
* Интерактивность (выборы с последствиями).
* Соответствие правилам построения сюжета.
Важно: Не добавляй пояснений – только чистый JSON!
"""

In [29]:
from httpx import ReadTimeout

In [30]:
# функция для генерации квеста
# Загружаем user_prompt
def generate_rpg_quest(user_prompt_txt_path, credentials):
    user_prompt = load_user_prompt(user_prompt_txt_path)

    try:
        giga = GigaChat(
            credentials=credentials,
            verify_ssl_certs=False,
            timeout=360
        )

        full_prompt = f"{SYSTEM_PROMPT}\n\nВходные данные:\n{user_prompt}\n\nВывод только в JSON!"

        response = giga.invoke(full_prompt)

        response_text = response.content

        try:
            quest_data = json.loads(response_text)
            return quest_data
        except json.JSONDecodeError as e:
            print(f"Ошибка парсинга JSON: {e}")
            print(f"Ответ модели: {response_text}")
            return {"error": "invalid_json"}

    except ReadTimeout:
        print("Ошибка: превышено время ожидания ответа от GigaChat.")
        return {"error": "timeout"}
    except Exception as e:
        print(f"Критическая ошибка: {str(e)}")
        return {"error": str(e)}


In [31]:
import json

In [32]:
if __name__ == "__main__":
    # Путь к файлу с user_prompt (например, "input.txt")
    USER_PROMPT_PATH = "/content/input.txt"  # Замените на ваш путь

    # Авторизационный ключ GigaChat
    CREDENTIALS = credentials  # Замените на реальный ключ

    # Генерация квеста
    quest = generate_rpg_quest(USER_PROMPT_PATH, CREDENTIALS)

    # Сохранение результата в файл
    with open("generated_quest.json", "w", encoding="utf-8") as f:
        json.dump(quest, f, indent=2, ensure_ascii=False)

    print("Квест успешно сгенерирован и сохранён в generated_quest.json!")

Квест успешно сгенерирован и сохранён в generated_quest.json!
